
1.   Aggiungi <unk> oov al tokenizer
2.   togli cose inutili
3.   provare .cache e .prefetch sul train e validation dataset con AUTOTUNE





# Constants

In [156]:
dataset_path = '/content/drive/My Drive/Challenge_3/anndl-2020-vqa.zip'
images_path = '/content/VQA_Dataset/Images/'
train_json_path = '/content/VQA_Dataset/train_questions_annotations.json'
test_json_path = '/content/VQA_Dataset/test_questions.json'
    
SEED = 1234

VALIDATION_SPLIT = 0.10

IMG_H = 400
IMG_W = 700

BATCH_SIZE = 25

classes = {
            '0': 0,
            '1': 1,
            '2': 2,
            '3': 3,
            '4': 4,
            '5': 5,
            'apple': 6,
            'baseball': 7,
            'bench': 8,
            'bike': 9,
            'bird': 10,
            'black': 11,
            'blanket': 12,
            'blue': 13,
            'bone': 14,
            'book': 15,
            'boy': 16,
            'brown': 17,
            'cat': 18,
            'chair': 19,
            'couch': 20,
            'dog': 21,
            'floor': 22,
            'food': 23,
            'football': 24,
            'girl': 25,
            'grass': 26,
            'gray': 27,
            'green': 28,
            'left': 29,
            'log': 30,
            'man': 31,
            'monkey bars': 32,
            'no': 33,
            'nothing': 34,
            'orange': 35,
            'pie': 36,
            'plant': 37,
            'playing': 38,
            'red': 39,
            'right': 40,
            'rug': 41,
            'sandbox': 42,
            'sitting': 43,
            'sleeping': 44,
            'soccer': 45,
            'squirrel': 46,
            'standing': 47,
            'stool': 48,
            'sunny': 49,
            'table': 50,
            'tree': 51,
            'watermelon': 52,
            'white': 53,
            'wine': 54,
            'woman': 55,
            'yellow': 56,
            'yes': 57
}

N_CLASSES = len(classes)



# Initial Setup



In [157]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [158]:
import os
from datetime import datetime
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from PIL import Image
import json
import math
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import cv2
import glob

import tensorflow_datasets as tfds

from imageio import imread

In [159]:
# Set the seed for random operations. 
# Letting experiments to be reproducible. 
tf.random.set_seed(SEED) 
np.random.seed(SEED)

# Get current working directory
cwd = os.getcwd()

In [160]:
# Mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [161]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jan  6 13:41:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    41W / 250W |  15523MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# RetriEval

In [162]:
! unzip '$dataset_path'

Output streaming troncato alle ultime 5000 righe.
  inflating: VQA_Dataset/Images/5390.png  
  inflating: VQA_Dataset/Images/5391.png  
  inflating: VQA_Dataset/Images/5392.png  
  inflating: VQA_Dataset/Images/5393.png  
  inflating: VQA_Dataset/Images/5394.png  
  inflating: VQA_Dataset/Images/5395.png  
  inflating: VQA_Dataset/Images/5396.png  
  inflating: VQA_Dataset/Images/5397.png  
  inflating: VQA_Dataset/Images/5398.png  
  inflating: VQA_Dataset/Images/5399.png  
  inflating: VQA_Dataset/Images/54.png  
  inflating: VQA_Dataset/Images/540.png  
  inflating: VQA_Dataset/Images/5400.png  
  inflating: VQA_Dataset/Images/5401.png  
  inflating: VQA_Dataset/Images/5402.png  
  inflating: VQA_Dataset/Images/5403.png  
  inflating: VQA_Dataset/Images/5404.png  
  inflating: VQA_Dataset/Images/5405.png  
  inflating: VQA_Dataset/Images/5406.png  
  inflating: VQA_Dataset/Images/5407.png  
  inflating: VQA_Dataset/Images/5408.png  
  inflating: VQA_Dataset/Images/5409.png  
  infla

# Tokenizer

In [163]:
def preprocess_question(question):
  question = question.replace("'", " '")
  question = question.replace("'monkey", "' monkey")
  question = question.replace("'t", "' t")
  question = question.replace("'mores", "' mores")
  question = question.replace("'more", "' more")  #'more after 'mores
  question = question.replace("'ing", "' ing")
  question = question.replace("'monkey", "' monkey")
  question = question.replace("'something", "' something")
  question = question.replace("'old", "' old")
  question = question.replace("'o", "' o")  # 'o must be put after 'old otherwise we would never reach 'old case
  question = question.replace("'jump", "' jump")
  question = question.replace("crossedq","crossed")
  question = question.replace("doorwh","door")
  return question  

In [164]:
def read_train_json():

  with open(train_json_path, 'r') as f:
    data = json.load(f)
  f.close()

  images = []
  questions = []
  answers = []

  for index in data:

    img_id = data[index]['image_id']

    question = data[index]['question']
    question = preprocess_question(question).split(" ")
    question[-1] = question[-1].replace('?', '')
    
    answer = data[index]['answer']

    images.append(img_id + '.png')
    questions.append(question)
    answers.append(classes[answer])

  TRAIN_SPLIT = math.ceil(len(images) * (1 - VALIDATION_SPLIT))
  train_images = images[:TRAIN_SPLIT]
  valid_images = images[TRAIN_SPLIT:]

  train_questions = questions[:TRAIN_SPLIT]
  valid_questions = questions[TRAIN_SPLIT:]

  train_answers = answers[:TRAIN_SPLIT]
  valid_answers = answers[TRAIN_SPLIT:]

  return train_images, train_questions, train_answers, valid_images, valid_questions, valid_answers

def read_test_json():

  with open(test_json_path, 'r') as f:
    data = json.load(f)
  f.close()

  ids = []
  images = []
  questions = []

  for index in data:
    
    img_id = data[index]['image_id']

    question = data[index]['question']
    question = preprocess_question(question)
    question = question.split(" ")
    question[-1] = question[-1].replace('?', '')
    
    ids.append(index)
    images.append(img_id + '.png')
    questions.append(question)

  return images, questions, ids

In [165]:
#extract images, questions and answer (or quest_id) from the train and test files
train_images, train_questions, train_answers, valid_images, valid_questions, valid_answers = read_train_json()
test_images, test_questions, questions_id = read_test_json()

In [166]:
word = "doorwh"
for question in train_questions:
  if word in question:
    print(question)

['What', 'color', 'is', 'the', 'doorwh']


In [167]:
# Use the Tokenizer to transform the text (questions) into sequence
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token = '<UNK>')

preprocessed_questions = []

with open(train_json_path, 'r') as f:

  data = json.load(f)

  for index in data:
    question = data[index]['question']
    question = preprocess_question(question)
    question = question.split(" ")
    question[-1] = question[-1].replace('?', '')
    #print(question)
    preprocessed_questions.append(question)

    # Updates internal vocabulary based on the questions of the dataset
    tokenizer.fit_on_texts(question)

f.close()

words_number = len(tokenizer.word_index) + 1 # + 1 is there to count the padding (element number 0)

In [168]:
with open(train_json_path, 'r') as f:
  data = json.load(f)
int(len(data) * (1 - VALIDATION_SPLIT))

52948

In [169]:
sequences = tokenizer.texts_to_sequences(train_questions)
max_length_train = max(len(sequence) for sequence in sequences)

sequences = tokenizer.texts_to_sequences(valid_questions)
max_length_valid = max(len(sequence) for sequence in sequences)

if max_length_valid > max_length_train:
  max_length = max_length_valid
else:
  max_length = max_length_train

train_input_questions = pad_sequences(sequences, maxlen=max_length)
valid_input_questions = pad_sequences(sequences, maxlen=max_length)

'''
tokenizer.fit_on_texts(test_questions)
sequences = tokenizer.texts_to_sequences(test_questions)
test_input_questions = pad_sequences(sequences, maxlen=max_length)

print()
print(len(train_images))
print(len(train_input_questions))
print(len(train_answers))

test_generator = DataGenerator(questions_id, test_images, test_input_questions,  max_length, to_fit=False, batch_size=1, dim=(IMG_H, img_w), n_classes=N_CLASSES, shuffle=False)
'''

'\ntokenizer.fit_on_texts(test_questions)\nsequences = tokenizer.texts_to_sequences(test_questions)\ntest_input_questions = pad_sequences(sequences, maxlen=max_length)\n\nprint()\nprint(len(train_images))\nprint(len(train_input_questions))\nprint(len(train_answers))\n\ntest_generator = DataGenerator(questions_id, test_images, test_input_questions,  max_length, to_fit=False, batch_size=1, dim=(IMG_H, img_w), n_classes=N_CLASSES, shuffle=False)\n'

In [170]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy()

# learning rate
lr = 5e-4
#optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr, rho=0.9)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model

#Embedding

In [171]:

embedding_path_zip = '/content/drive/My Drive/Challenge_3/glove.6B.300d.txt.zip'
!unzip '$embedding_path_zip'


unzip:  cannot find or open /content/drive/My Drive/Challenge_3/glove.6B.300d.txt.zip, /content/drive/My Drive/Challenge_3/glove.6B.300d.txt.zip.zip or /content/drive/My Drive/Challenge_3/glove.6B.300d.txt.zip.ZIP.


In [172]:
path_to_glove_file = os.path.join("/content/glove.42B.300d.txt")

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

# ------------------------------------------------------------


FileNotFoundError: ignored

In [173]:
embedding_dim = 300
hits = 0
misses = 0

missing_words = []

# Prepare embedding matrix
embedding_matrix = np.zeros((words_number, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        print(word)
        misses += 1
        missing_words.append(word)
print("Converted %d words (%d misses)" % (hits, misses))

In [174]:
tokenizer.word_index

In [175]:
def compute_embedding(missing_word):
    count = 0
    weights = np.zeros(embedding_dim)
    #print(weights)
    for question in train_questions:
      if missing_word in question:
        print(question)
        
        for word_question in question:
          if word_question not in missing_words:
            print(word_question)
            embedding_vector = embeddings_index.get(word_question.lower())
            #print(word + " rank:%s" % i)
            if embedding_vector is not None:
               #print(embedding_vector)
               weights = weights + embedding_vector
               count = count +1
            else :
               print("Error")
    weights = weights / count
    

    return weights


# Dataset Setup

In [176]:
# -------------------------------------------------------------
# Create the training and validation ImageDataGenerator objects
# -------------------------------------------------------------
def data_generator(apply_data_augmentation = True):

  from tensorflow.keras.preprocessing.image import ImageDataGenerator

  apply_data_augmentation = True

  # Create training ImageDataGenerator object
  # We need two different generators for images and corresponding masks
  if apply_data_augmentation:
    img_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')
    
  return img_data_gen

In [177]:
# --------------------------------
# Create the class for the dataset
# --------------------------------

from PIL import Image

class TrainDataset(tf.keras.utils.Sequence):

  """
    CustomDataset inheriting from tf.keras.utils.Sequence.

    3 main methods:
      - __init__: save dataset params like directory, filenames..
      - __len__: return the total number of samples in the dataset
      - __getitem__: return a sample from the dataset

    Note: 
      - the custom dataset return a single sample from the dataset. Then, we use 
        a tf.data.Dataset object to group samples into batches.
      - in this case we have a different structure of the dataset in memory. 
        We have all the images in the same folder and the training and validation splits
        are defined in text files.

  """

  def __init__(self, tokenizer, set_type, img_generator=None, preprocessing_function=None, img_shape=[400, 700]):
    import json
    
    with open(train_json_path, 'r') as f:
      data = json.load(f)
    
    # Tokenizer fuori perche deve creare variabili globali come word_number ecc.

    self.data = {}
    for i, j in enumerate(data):

      if set_type == 'training':

        if i + 1 <= int(len(data) * (1 - VALIDATION_SPLIT)):
          self.data[i] = data[j]
        else:
          break
      
      else:
        
        if i + 1 > int(len(data) * (1 - VALIDATION_SPLIT)):
          self.data[i-int(len(data) * (1 - VALIDATION_SPLIT))] = data[j] # self.data[i-int(len(data) * (1 - VALIDATION_SPLIT))] to obtain from index 0
        else:
          continue
        

    self.tokenizer = tokenizer
    
    self.img_generator = img_generator
    
    self.preprocessing_function = preprocessing_function
    
    self.img_shape = img_shape

    self.SEED = 1234

  def __len__(self):
    return len(self.data)
  
  def on_epoch_end(self):

    new_indexes = range(len(self.data))
    np.random.shuffle(new_indexes)
    
    new_data = {}
    for i, j in enumerate(new_indexes):
      new_data[i] = self.data[j]
    
    self.data = new_data

  def __getitem__(self, index):

    curr_element = self.data[index]

    # Image
    # -----
    
    # Read Image
    #img = Image.open(images_path + curr_element['image_id'] + '.png').convert('RGB')
    img = cv2.imread(images_path + curr_element['image_id'] + '.png', cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB ) 
    # Resize image
    img = cv2.resize(img, (self.img_shape[1], self.img_shape[0]), interpolation = cv2.INTER_AREA) #size – The requested size in pixels, as a 2-tuple: (WIDTH, HEIGHT). ! ATTENTION ! cv2 function
    
    if self.img_generator is not None:
      # Perform data augmentation
      # We can get a random transformation from the ImageDataGenerator using get_random_transform
      # and we can apply it to the image using apply_transform

      #self.SEED = self.SEED + 1
      img_t = self.img_generator.get_random_transform(img.shape, seed=self.SEED)
      img = self.img_generator.apply_transform(img, img_t)
    
    if self.preprocessing_function is not None:
      img = self.preprocessing_function(img)

    # -----

    # Question
    # --------

    question = curr_element['question']
    question = preprocess_question(question)
    question = question.split(' ')
    question[0]= question[0].replace('?', '')
    #print(question)
    question = self.tokenizer.texts_to_sequences(question)
    #print(question)
    question = pad_sequences(question, padding = 'post', maxlen = max_length)
    question = question[0]
    # --------

    # Answer
    # ------

    answer = curr_element['answer']
    #print(answer)
    answer = classes[answer]
    #print(answer)
    # One hot encoding????????????????????????????????????????????????????????????? serve????????????????'
    indices = range(N_CLASSES)
    depth = N_CLASSES
    answer = tf.one_hot(indices, depth)[answer].numpy()

    # ------

    return (np.float32(img), np.int32(question)), np.int32(answer)

In [178]:
# ------------------------------------------------------------------
# Create train and validation dataset objects using the custom class
# ------------------------------------------------------------------
def create_dataset(img_data_gen, img_h, img_w, bs, num_classes):

  # Training
  # --------
  train_gen = TrainDataset(tokenizer = tokenizer,
                           set_type = 'training',
                           #img_generator = data_generator(), 
                           img_shape = [img_h, img_w], # size – The requested size in pixels, as a 2-tuple: (h, w). !! np function
                           preprocessing_function = None
                          )
  
  train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                                 output_types = ((tf.float32, tf.int32), tf.int32),
                                                 output_shapes = (([img_h, img_w, 3], [max_length]), N_CLASSES))
                                                                                            
  train_dataset = train_dataset.batch(bs)

  train_dataset = train_dataset.repeat()
  # --------

  # Training
  # --------
  valid_gen = TrainDataset(tokenizer = tokenizer,
                           set_type = 'validation',
                           img_generator = None, 
                           img_shape = [img_h, img_w], # size – The requested size in pixels, as a 2-tuple: (h, w). !! np function
                           preprocessing_function = None
                          )
  
  valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                                 output_types = ((tf.float32, tf.int32), tf.int32),
                                                 output_shapes = (([img_h, img_w, 3], [max_length]), N_CLASSES))
                                                                                            
  valid_dataset = valid_dataset.batch(bs)

  # --------

  print('Training samples: ' + str(len(train_gen)))
  print('Validation samples: ' + str(len(valid_gen)))

  return train_dataset, train_gen, valid_dataset, valid_gen

In [179]:
train_dataset, train_gen, valid_dataset, valid_gen = create_dataset(img_data_gen = None, img_h = IMG_H, img_w= IMG_W, bs = BATCH_SIZE, num_classes= N_CLASSES)

train_dataset = train_dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

In [180]:
for word, i in train_gen.tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        print(word)
        misses += 1
        missing_words.append(word)
print("Converted %d words (%d misses)" % (hits, misses))
  

In [181]:
from google.colab.patches import cv2_imshow

for (image, question), answer in train_dataset:
  print(image.shape)
  cv2_imshow(np.array(image[0]))
  question[0]
  answer[0]    
  break

# Training

In [182]:
# -----------------
# Set the callbacks
# -----------------
def set_callbacks(type = 'transfer_learning', model_checkpoint = True, visualize_tensorboard = False, early_stop = True, reduce_lr_on_plateau = True):
  
  import os
  from datetime import datetime

  cwd = os.getcwd()

  exps_dir = os.path.join('/content/drive/My Drive/Challenge_3/', 'classification_experiments')
  if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

  model_name = type

  exp_dir = os.path.join(exps_dir, model_name)
  if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
  callbacks = []

  # Model checkpoint
  # ----------------
  if model_checkpoint:
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                       save_weights_only=True,
                                                       save_best_only=True)  # False to save the model directly
    callbacks.append(ckpt_callback)
  # ----------------

  # Visualize Learning on Tensorboard
  # ---------------------------------
  if visualize_tensorboard:
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
    
    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)  # if 1 shows weights histograms
    callbacks.append(tb_callback)
  # ---------------------------------

  # Early Stopping
  # --------------
  if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                   patience = 2,
                                                   restore_best_weights=True)
    callbacks.append(es_callback)
  # --------------
  '''
  # Reduce Learning rate on plateau
  # -------------------------------
  if reduce_lr_on_plateau:
    rlr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                        factor=0.5, 
                                                        patience=1, 
                                                        verbose=1, 
                                                        mode='auto', 
                                                        min_delta=0.0001)
    callbacks.append(rlr_callback)
  # -------------------------------
  '''
  return callbacks

In [183]:
'''
! wget http://nlp.stanford.edu/data/glove.42B.300d.zip
! unzip -q glove.42B.300d.zip
'''

In [184]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length = maxlen, 
                                                   embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix), mask_zero=True, trainable = False)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim, mask_zero=True)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [185]:
CNN_Input = tf.keras.layers.Input(shape=(IMG_H,IMG_W,3), name='image_input')

#mobilenetv2 = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(IMG_H,IMG_W,3), alpha=1.0, include_top=False, weights='imagenet', input_tensor=CNN_Input)

efficient = tf.keras.applications.EfficientNetB7(input_shape=(IMG_H,IMG_W,3), include_top=False, weights='imagenet', input_tensor=CNN_Input)

efficient.trainable = True

for layer in efficient.layers:
  layer.trainable = False
  
  #if ("block4" in layer.name):
  #  layer.trainable = True
  #if ("block5" in layer.name):
    # layer.trainable = True
  #if ("block6" in layer.name):
  #  layer.trainable = True
  if ("block7" in layer.name):
    layer.trainable = True
  if "_bn" in layer.name:
    layer.trainable = False
  if 'top_conv' == layer.name:
    layer.trainable = True
  if 'top_activation' == layer.name:
    layer.trainable = True
  
  # Print the layer of the EfficientNet-B7 and if they are set to trainable
  #print(str(layer.name) + ' '*(35 - len(str(layer.name))) + '----> Trainable: ' + str(layer.trainable))

#efficient.summary()

CNN_model=tf.keras.models.Sequential()
CNN_model.add(CNN_Input)
CNN_model.add(efficient)
CNN_model.add(tf.keras.layers.GlobalAveragePooling2D())

#shape = tf.shape(CNN_model.output)
#OUT = tf.reshape(CNN_model.output, (-1, 286, shape[3]))

#OUT = tf.reduce_mean(CNN_model.output, 1)

#OUT = TransformerBlock(2560, 8, 64)(OUT)
#OUT = tf.keras.layers.GlobalAveragePooling1D()(OUT)

#Creating the RNN model for text processing

RNN_model = tf.keras.models.Sequential()
RNN_Input = tf.keras.layers.Input(shape = (max_length), name='text_input')
RNN_model.add(RNN_Input)

'''
#RNN_model.add(tf.keras.layers.Embedding(input_dim = words_number, output_dim = 2048, input_length = max_length, mask_zero=True))

# GloVe 100 dimensional embeddings
RNN_model.add(tf.keras.layers.Embedding(input_dim = words_number, output_dim = embedding_dim, input_length = max_length, 
                                        embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix), mask_zero=True, trainable = False))

RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, stateful=False, return_sequences=True, recurrent_initializer='glorot_uniform')))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, stateful=False, return_sequences=False, recurrent_initializer='glorot_uniform')))
#RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, stateful=False, return_sequences=True, recurrent_initializer='glorot_uniform')))
#RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, stateful=False, return_sequences=False, recurrent_initializer='glorot_uniform')))
'''

embed_dim = 300  # Embedding size for each token
num_heads = 10  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer

RNN_model.add(TokenAndPositionEmbedding(max_length, words_number, embed_dim))
'''
OUT = tf.keras.layers.Dense(1024, activation='relu')(CNN_model.output)

OUT = tf.keras.layers.Multiply()([tf.expand_dims(OUT, axis = 1), TOKEN_model.output])

OUT = TransformerBlock(embed_dim, num_heads, ff_dim)(OUT)

OUT = tf.keras.layers.GlobalAveragePooling1D()(OUT)
'''

RNN_model.add(TransformerBlock(embed_dim, num_heads, ff_dim))
RNN_model.add(tf.keras.layers.GlobalAveragePooling1D())


concat = tf.keras.layers.concatenate([CNN_model.output, RNN_model.output])

# Added ----------------------------------------------------------------------------------------
dense_out = tf.keras.layers.Dense(512, activation='relu', 
                                  #kernel_regularizer = tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
                                  #bias_regularizer = tf.keras.regularizers.l2(1e-4),
                                  activity_regularizer = tf.keras.regularizers.l2(1e-5))(concat)
#dense_out = tf.keras.layers.Dense(128, activation='relu')(dense_out)
tf.keras.layers.Dropout(0.2, seed=SEED)

dense_out = tf.keras.layers.Dense(N_CLASSES, activation='softmax', name='output')(dense_out)

model = tf.keras.Model(inputs=[CNN_Input, RNN_Input],
                       outputs = dense_out)

def scheduler(epoch, lr):
  if epoch == 1:
    return lr
  else:
    return lr * 0.1 

model.compile(
  tf.keras.optimizers.Adam(lr=1e-4), # somewhat arbitrarily chosen
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

model.summary()

callbacks = set_callbacks(type='effB7_finetune7_Transformer_1024x8_ff=64_512fc_dropout20_new_way')
#callbacks.append(tf.keras.callbacks.LearningRateScheduler(scheduler))

tf.keras.utils.plot_model(model)

#Execution

In [ ]:
model.fit(train_dataset,
          validation_data = valid_dataset,
          steps_per_epoch = len(train_gen)//BATCH_SIZE,
          epochs = 100,
          callbacks = callbacks)

Epoch 1/100
1405/2117 [==================>...........] - ETA: 15:34 - loss: 1.1754 - accuracy: 0.5440

# Nuova sezione

In [187]:
exps_dir = os.path.join('/content/drive/My Drive/Challenge_3/', 'classification_experiments')
model_name = 'effB7_finetune7_Transformer_1024x8_ff=64_512fc_dropout20_new_way'
exp_dir = os.path.join(exps_dir, model_name)
ckpt_dir = os.path.join(exp_dir, 'ckpts')
latest = tf.train.latest_checkpoint(ckpt_dir)
print(latest)
model.load_weights(latest)

# Results (csv)

In [ ]:
# Create folder for prediction files (csv)
! mkdir '/content/drive/My Drive/Challenge_3/result_csv'

# ------------------------------------
# Create the csv file with predictions
# ------------------------------------
def create_csv(results, results_dir='/content/drive/My Drive/Challenge_3/result_csv'):
  import os
  from datetime import datetime

  csv_fname = 'results_'
  csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

  with open(os.path.join(results_dir, csv_fname), 'w') as f:
    f.write('Id,Category\n')
    for key, value in results.items():
      f.write(key + ',' + str(value) + '\n')

In [ ]:
! ls '/content/drive/My Drive/Challenge_3/result_csv'

In [ ]:
# --------------------------------
# Create the class for the dataset
# --------------------------------

from PIL import Image

class TestDataset(tf.keras.utils.Sequence):

  """
    CustomDataset inheriting from tf.keras.utils.Sequence.

    3 main methods:
      - __init__: save dataset params like directory, filenames..
      - __len__: return the total number of samples in the dataset
      - __getitem__: return a sample from the dataset

    Note: 
      - the custom dataset return a single sample from the dataset. Then, we use 
        a tf.data.Dataset object to group samples into batches.
      - in this case we have a different structure of the dataset in memory. 
        We have all the images in the same folder and the training and validation splits
        are defined in text files.

  """

  def __init__(self, tokenizer, img_generator=None, preprocessing_function=None, img_shape=[400, 700]):
    import json
    
    with open(test_json_path, 'r') as f:
      data = json.load(f)

    # Tokenizer fuori perche deve creare variabili globali come word_number ecc.

    self.data = {}
    for i, j in enumerate(data):
      self.data[i] = data[j]
      self.data[i]['question_id'] = j

    self.tokenizer = tokenizer
    
    self.img_generator = img_generator
    
    self.preprocessing_function = preprocessing_function
    
    self.img_shape = img_shape

    self.SEED = 1234

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):

    curr_element = self.data[index]

    # Image
    # -----

    question_id = curr_element['question_id'] #################################################################################
    
    # Read Image
    img = cv2.imread(images_path + curr_element['image_id'] + '.png', cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB ) 
    # Resize image
    img = cv2.resize(img, (self.img_shape[1], self.img_shape[0]), interpolation = cv2.INTER_AREA)
    
    if self.img_generator is not None:
      # Perform data augmentation
      # We can get a random transformation from the ImageDataGenerator using get_random_transform
      # and we can apply it to the image using apply_transform

      #self.SEED = self.SEED + 1
      img_t = self.img_generator.get_random_transform(img_arr.shape, seed=self.SEED)
      img_arr = self.img_generator.apply_transform(img_arr, img_t)
    
    if self.preprocessing_function is not None:
      img_arr = self.preprocessing_function(img_arr)

    # -----

    # Question
    # --------

    question = [curr_element['question']]#.split(' ')
    question[0]= question[0].replace('?', '')
    #print(question)
    question = self.tokenizer.texts_to_sequences(question)
    #print(question)
    question = pad_sequences(question, padding = 'post', maxlen = max_length)
    question = question[0]
    # --------

    return question_id, (np.float32(img), np.int32(question))

In [ ]:
# ------------------------------------------------------------------
# Create train and validation dataset objects using the custom class
# ------------------------------------------------------------------
def create_test_dataset(img_data_gen, img_h, img_w):

  # Training
  # --------
  test_gen = TestDataset(tokenizer = tokenizer,
                        img_generator = None, 
                        img_shape = [img_h, img_w], # size – The requested size in pixels, as a 2-tuple: (h, w). !! np function
                        preprocessing_function = None
                        )

  test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                                 output_types = (tf.string, (tf.float32, tf.int32)),
                                                 output_shapes = (None, ([img_h, img_w, 3], [max_length])))
                                                                                            
  test_dataset = test_dataset.batch(1)
  # --------
  
  return test_dataset, test_gen

In [ ]:
test_dataset, test_gen = create_test_dataset(img_data_gen = None, img_h = IMG_H, img_w = IMG_W)

In [ ]:
for i, (j,k) in test_dataset:
  id  = np.array_str(i.numpy())[3:-2]
  id
  break

In [ ]:
# --------------------------------------
# Make predictions and save the csv file
# --------------------------------------
from PIL import Image

results = {}
counter = 0

for question_id, input in test_dataset:

  counter += 1
  
  question_id = np.array_str(question_id.numpy())[3:-2]

  out_softmax = model.predict(x=input)

  prediction = tf.math.argmax(out_softmax, 1)

  results[question_id] = prediction[0].numpy() # Get first of the batch (but bs = 1)

  print(question_id + ' ---> ' + str(results[question_id]))
  print('------------------------------------------------> ' + str(counter) + ' / ' + str(len(test_gen)))

create_csv(results)

! ls '/content/drive/My Drive/Challenge_3/result_csv'